<a href="https://colab.research.google.com/github/GoodLight999/webui_colab/blob/main/webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

・このノートは、camenduru氏の配布しているipynbを参考にして作られています。</br>
https://github.com/camenduru/stable-diffusion-webui-colab</br>
</br>
・権利者に無断で、画像生成AIにイラストを学習させる行為は、日本の著作権法30条の4第2号に則って完全に**合法**です。また、拡散モデルによる画像生成は、ニューラルネットを用いたノイズ除去と画像分類が元になっており、**イラストのコラージュからは程遠い手法**です。しかしそれはそれとして**不快に思うイラストレーター**は居ます。</br>
また、そういった一部の人の気持ちを気にしなくとも、現在の高品質なアニメ調イラスト生成モデルには、NovelAIから**不正流出したデータを元に作られている限りなく黒に近いモデルが氾濫しているという非常に大きな問題**があります。</br>
加えて、danbooruという英語による丁寧なタグ付けがなされた**イラスト転載サイトを学習元に使っているモデルも多い**です。法的にはクリーンなWaifu Diffusion もこの問題からは逃れられません。</br>
danbooru自体は、転載元をしっかり掲載したうえで画像を詳細にタグ付けしている非営利のサイトで、米国法で無断転載が合法となるフェアユース規定に則ろうとしている(比較的)誠実な転載サイトではあります。</br>
　(フェアユースの例：Google画像検索の検索結果には、無断転載された画像が多数並んでいるが、合法とされている)</br>
でも、無断転載であることに不快感を覚えるイラストレーターが居るのは事実。</br>
こういったことを念頭に置いて、画像を生成しましょう。</br>
</br>
・たぶん、</br>
**編集　→ノートブックの設定　→ハードウェア アクセラレータ　→GPU**</br>
と設定したあと</br>
**ランタイム　→全てのセルを実行**</br>
を押すだけで動きます！</br>
起動にはけっこう時間がかかります。8分近くかかるかも……</br>
PC推奨です。処理はGoogleのサーバーを借りて行うので、一応スマホからでも動かせはします。ちょっと使いにくいけどね。</br>
加えて、長時間使っていると、強制的に切断させられます。無料だから仕方がないね。

In [ ]:
#@title 起動準備
# aria2(超高速ダウンロードを並列化で実現する仕組み)を導入
!apt -y install -qq aria2

# 画像生成高速化の仕組みを準備
%cd /content
import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16
!pip install -q triton==2.0.0

In [ ]:
#@title webuiの準備
# webuiを連れてくる
!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui

!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py

In [ ]:
#@title webui向けの拡張機能・追加ファイルの導入
# image briwser
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
# civitai browser
# !git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
# additional networks(for LoRA)
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
# tag complete
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
# aspect ratio selector
!git clone https://github.com/alemelis/sd-webui-ar /content/stable-diffusion-webui/extensions/sd-webui-ar
# ControlNet
!git clone https://github.com/Mikubill/sd-webui-controlnet.git /content/stable-diffusion-webui/extensions/sd-webui-controlnet
# Openpose editor (for ControlNet)
!git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
# 日本語化
!git clone https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP
!git clone https://github.com/journey-ad/sd-webui-bilingual-localization /content/stable-diffusion-webui/extensions/sd-webui-bilingual-localization
# sdweb-easy-prompt-selector
!git clone https://github.com/blue-pen5805/sdweb-easy-prompt-selector /content/stable-diffusion-webui/extensions/sdweb-easy-prompt-selector
# sd-webui-tunnels
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
# multidiffusion-upscaler-for-automatic1111
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/stable-diffusion-webui/extensions/sd-webui-tunnels


# EasyNegative
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors
# EasyNegative
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt
# bad_prompt_version2.pt
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt
# wdbadprompt.pt
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/waifu-diffusion/wd-1-5-beta/resolve/main/embeddings/wdbadprompt.pt
# wdgoodprompt.bin
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/waifu-diffusion/wd-1-5-beta/resolve/main/embeddings/wdgoodprompt.bin


# 生成完了時に通知音を鳴らすようにする
!wget https://otologic.jp/sounds/se/pre/Accent03-1.mp3 -O /content/stable-diffusion-webui/notification.mp3

# ディレクトリを移動
%cd /content/stable-diffusion-webui

!git reset --hard

In [ ]:
#@title 背景を詰め合わせたyamlファイルの用意
import yaml

# 背景用yamlファイルの内容
yaml_content = '''
背景コレクション50:
  難破船: (crystal clear water:1.4),(anchor:1.4),(blue sky:1.5),(Shipwrecks:1.4),cumulus,
  図書館: (((((messy room))))),((((((gothic room)))))),((dark atmosphere)),(((canopy))),clutter,(((lots of colorful books:1.5))),((Miscellaneous goods are placed in a mess:1.5)),(((colorful sphere))),(((Stained glass:1.3))) ,
  お部屋: (in the messy room:1.3),(cute interior:1.5),(pink room),window,kawaii room,canopy,frills,pink chair,(baldachin:1.5),(Foliage plant:1.2),(chesterfield chair:1.5),
  街歩き: (walking in the town:1.5),(outdoor:1.5),(in street:1.5),Cafeteria, Florist, Shoemaker, Florist,
  海の中: ((zero gravity,weightlessness,flying,floating in the air)),((arbor)) ,wonderland,((tropical fish, flock of tropical fish, colorful tropical fish, underwater, in the sea,school of fish, seaweed,jellyfish,pillar of light)), ((large shellfish,coral reef,airbladder:1.5 )), (((larg anchor,Deep sea background ))),goddess,((((large amount of fish)))),((garden)),Visual impact,
  コウモリと月: (((Visual impact,flying shadow))),((((flying bats, bats shadow,grow pink moon)))),((round frame:1.3)),full moon,(City of night,in street),(lamp),(large gate:1.3),
  ワンダーランドガーデン: ((yumekawa,yumekawaii,pastel color)),(((wonderland garden:1.3))),rainbow waterfall,(((dripping ink))),imagination,(Castle:1.4),
  雲の上の街: (above the clouds:1.4), (in the sky:1.3) , (large full moon:1.2), round frame background,circle frame background, best illuminate,wonderland,(some floating island,floating city:1.5),(floating girl:1.5), weightlessness,zero gravity ,giant scales,
  かわいいるーむ: (((( pink kawaii room )))),,((((pink gothic room)))),(((messy room, Marchen ,macarons,Candy,heart item,ribbon,star item, small window))),((Miscellaneous goods are placed in a mess:1.5)),dresser,antimacassar ,
  未来と鳥居: (cyberpunk:1.5),(biopunk:1.5),(cyberpunk city:1.5),(holographic billboard:1.5),(neon signs:1.3),(City of night:1.5),(utility pole with power line:1.3) ,(torii:1.2),(shrine:1.2), (Japanese arch:1.2),huge building,
  雨上がりの夕焼け: cityscape,dusk, gradient sky,horizon,(mountainous horizon:1.1),orange sky, sunlight, sunrise, sunset,(tree:1.3),(Sunset reflected in a puddle:1.4),(in the forest:1.5),(puddle:1.3), after rain,
  歯車とマシン: (many gears:1.3),(one huge gear:1.5),filigree and gorgeous,in the dark night,(huge gear halo:1.3),(facing forward:1.5),(The center of the gear that confined the universe:1.3),(big gear machine:1.5),glitch art,revolve round ,circle,
  エフェクト宇宙: Visual impact ,(spooky:1.5),imagination,(glitch art:1.3),(revolve round:1.3),fluorescent ,( Fibonacci spiral :1.3),(galaxy:1.5),(extreme detailed effect:1.5),(Lightning and body Interweave:1.5),(milky way:1.3),(goddess:1.5),
  エフェクト炎: (dark flame:1.3),burn circle,((clear and heavy)),((burn the sky, vortex of fire, fire magic:1.5)), hand of fire,((melt and mix:1.8)), ((flame and burning:1.3)),black fog,wind,((( fusion of flame and hair:1.3))),weightlessness,eddy, (burn eddy effect:1.5),(round flame background:1.5),(round frame:1.4),(huge halo:1.4),(flame blast:1.3),
  黒ゴス: Horizontal view,(gothic room:1.3),(sitting on the gothic sofa:1.3),(black sofa:1.2),front angle,dark intense shadows ,( deep dark:1.5),dark room,(gothic :1.5),(black and white window:1.3),(chessboard floor:1.3),(darksome:1.5),(baddest:1.5),(gothic Lantern:1.5),(large clock:1.3),spooky,bloom,(black interior:1.2),
  クラゲたち: colorful,(flying jellyfish:1.5),standing in the ocean,(blue sky:1.2),(large amount of jellyfish:1.3),(Mysterious:1.3),(luminescence:1.3),
  高所と街並み: (summer sky:1.3),(cumulonimbus),blue sky,group of buildings,1Contrail,(cityscape:1.5),warm light, the top of the hill,(Overlooking the cityscape:1.5),(scenery:1.5),(Looking down on the city from the roof of a building:1.3),
  室内ガーデン: (Foliage plant:1.3) , ((fountain,water:1.3)), luster,Flower, rose Garden,indoor, (alcove , arbor:1.2) ,room ,Garden room,open window,(lamp),rose,(visual Impact:1.5),splash Flower, (crystal clear water:1.4),(botanical garden:1.3),small waterfall,
  汚部屋: (Miscellaneous goods are placed in a mess,messy room:1.8),
  サイバーパンク: (cyberpunk city:1.8),(biopunk girl:1.5),(light rays:1.2) ,(milky way:1.3),(((luminescence))),dark atmosphere,(neon sign:1.3),skyscraper,bloom,(City of night:1.3),
  骸部屋: (from above:1.3),(in dark shadow room:1.8),(dark),(large bird cage:1.5),(thorns:1.5),(Rose:1.5),(thorns effect:1.5),(spooky,imagination,baddest,huge skull:1.5),(darksome:1.5),(lamp:1.2),(skull gargoyle statue:1.5),(spooky wall clock:1.2),(red carpet,big old mirror:1.2),
  キラキラとハートの夜: ((wonderland)),Gothic decoration,((sticker style)),(heart item,star item,ribbon:1.5),(symbolic:1.2), conspicuous,(star glitter and hart glitter:1.5),(kawaii town at night:1.5),(big clock tower:1.2),(Meteor shower,star and hart effect:1.3),
  夏祭り: (milky way),(giant paper lantern:1.2),(stone lantern),(fireworks:1.3),(crowd,stall:1.3),
  戦場化した町: (☢:1.3),(smoke:1.2),(combat,wrecked tank:1.4),(in ruins,in town),(outdoor,sky), (sparks:1.3),(wartime:1.2),
  バー: (in bar),(Various colored bottles lined up on the counter:1.5),(indirect lighting,bottles),(Lounges, dark bars),(Cocktail:1.2),(bar counter:1.3),(luster),
  黄昏みなと: (Many small boats are parked in the harbor where you can see the sky and cumulonimbus clouds:1.3),(piers:1.3),seagulls flying in the distance,twilight,
  露店と城下町: (fantasy,castle and town:1.5),(castle town,beautiful building:1.3),(how people live),(flowers, trees, nature:1.3),romantic,(shopping street,street stall:1.2),(blue sky),
  木漏れ日と川: (river surrounded by trees:1.2),blue sky and cumulonimbus,(camping:1.3),(Riverside:1.3),(in shadow,arbor,light reflecting in the river,tree shade, sunbeams:1.5),
  月と光の蝶: ((tile floor)),((pink moon:1.3)), (🦋,some butterflies are flying,flying glowing butterfly:1.3),(night sky ,a lot of space:1.3),
  工場: (Factories, pipes, smoke:1.3),(sparks, chimneys, electric lights, rusty walls:1.3),(outdoor),(a large oil refinery with a lot of tanks and pipes:1.3),(factory view at night),
  春と和室: (Japanese-style room, porch,japanese garden view:1.3),(tatami:1.2),bonsai,(Fusuma),(cherry blossoms:1.3),bloom,
  ピンクローズ: (thorns:1.1),(lots of roses:1.5),(pink rose:1.5),,(chessboard,fantasy),((pink gothic room:1.5)),((pink tile floor,tile floor:1.2)),(sweets decoration),(white and pink interior,sweets:1.3),
  マカロンエフェクト: (white background:1.3),(wonderland), (Minimalism:1.5),(lots of macarons background:1.5),(Candy,cake,heart,star:1.3),(Large amount of macarons:1.3),(sweets and kawaii and cute:1.3),(magical effect:1.3),
  トロピカル: (beach, palm trees:1.3), beach, clear water,(blue sky, cumulonimbus:1.3),(starfish, seashell:1.2),(island),(hibiscus:1.4),
  遊園地: (Amusement park:1.5),(attractions, theme parks,roller coaster:1.2),(ferris wheel:1.2),(flower,fountain:1.3),(outdoor:1.3),(blue sky:1.3),
  エフェクト雷: (electric current,Magic Circle,Explosive lightning:1.5),(revolve round:1.5),(goddess:1.3),(lightning effect:1.5),
  教会: (fantasy),(in church,large cross:1.5),(baroque:1.5),God Light,(intricate),Stained glass,skylight,(divine,Holy,goddess:1.3),
  宇宙開発: (milky way:1.3),(universe,spacecraft:1.5),(earth,container, cable,small drone,factory,weightless, floating:1.5)(outer space,Artificial satellite,space station,space colony:1.3),(asteroid,Accretion Disk:1.3),
  きのこの国: (in forest:1.5), (giant mushrooms, many mushrooms:1.3), (glowing mushrooms:1.3),(starry sky,night:1.3), moon,(fantastic sight:1.3),fire Fly,(big mushroom forest:1.3),
  アウトローシティー: (back alley:1.5), bricks,(gangland,wall text graffiti:1.5),(messy town), gang, shutters,darksome,spooky,(dangerous atmosphere,badness,midnight,night, night road:1.5),fluorescent lamp,
  田舎の駅: (retro chic,Battered train station,Old station:1.4), (wooden building:1.2), countryside scenery,(on station platform,railroad tracks:1.3),(outdoor,blue sky,cumulonimbus:1.1),
  水晶神殿: (water temple,crystal clear water:1.3),(crystal, reflective, glowing, waterlogged floor,fountain:1.5),(waterfall),(crystal textured interior),(water ripples:1.3),(clear blue crystal pillars:1.2),(geyser,indoor),(ice floor, ice wall:1.2),(light blue glowing:1.3),
  風車と花畑: (in Flower garden, colorful flower garden,landscape of flowers:1.5),( blue sky, cumulonimbus), (windmill:1.3),(wooden bench:1.2),(many flowers are blooming:1.3),
  文明の果て: (broken glass, peeled walls,jungle, city,city center, building,moss, ivy:1.4),( blue sky, cumulonimbus),(collapsed town,broken town, deserted town,roots crawling on walls),(outdoor),(rubble,ruins:1.3),
  夜桜と日本の城: (Japanese castle, cherry blossoms, banquet, night, starry sky:1.3),(full moon,Japanese castle lit up:1.2),(cherry blossom petals:1.4),
  ナイトプール: (luminous water,balcony,jacuzzi pool,night pool, light up, tile floor:1.5),(crystal clear water),(superb view,night view,Night view with skyscrapers:1.1),(outdoors,starry sky),(indirect lighting:1.3),
  山粧う: (Giant trees, leaves that cover the sky:1.3),(maple leaves,Autumn leaves, autumn,red mountain,Fallen leaves:1.1),waterfall,
  雪の町: (town, snow, snow city, snowing:1.5),(1snowman:1.1),(residential area, street corner:1.3),(outdoors),
  雨の日: (heavy rain,raining,pouring rain:1.5),( puddles, water ripples:1.2),(outdoor,in street:1.3),(cloudy),(in front of the shop:1.4),
  虹夢: (Wonderland, Fantasy,:1.3),(Rainbow,flower tree,sweets Clouds:1.5),(yumekawa,yumekawaii,pastel color:1.3),(sweets garden:1.3),(land of sweets,confections,rainbow arch:1.3),(Sugar confectionery,round rainbow frame:1.3),bloom,blue sky,

解説サイト:
  作者様解説（画像付き）: https://grm.chichi-pui.com/articles/event/aicollection/hpa3n41o4lqi
  このファイル配布元: https://sp8999.com/stable-diffusion/2023/03/26/1023/
'''

# ファイルパス
dir_path = "/content/stable-diffusion-webui/extensions/sdweb-easy-prompt-selector/tags"
file_name = "背景コレクション50.yml"
file_path = os.path.join(dir_path, file_name)

# ディレクトリが存在しない場合には、作成する
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# ファイルを開いて内容を書き込む
with open(file_path, "w") as f:
    f.write(yaml_content)

print(f"ファイルを保存しました: {file_path}")


In [ ]:
#@title ネガティブプロンプトを詰め合わせたyamlファイルの用意
import yaml

# 背景用yamlファイルの内容
yaml_content = '''
ネガティブプロンプト:
  AOM向けネガティブプロンプト: |
    EasyNegative, (worst quality, low quality:1.4), (lip, nose, tooth, rouge, lipstick, eyeshadow:1.4), (blush:1.2), (jpeg artifacts:1.4), (depth of field, bokeh, blurry, film grain, chromatic aberration, lens flare:1.0), (1boy, abs, muscular, rib:1.0), greyscale, monochrome, dusty sunbeams, trembling, motion lines, motion blur, emphasis lines, 
    BREAK 
    (text, title, logo, signature:1.1), manga
'''

# ファイルパス
dir_path = "/content/stable-diffusion-webui/extensions/sdweb-easy-prompt-selector/tags"
file_name = "ネガティブプロンプト.yml"
file_path = os.path.join(dir_path, file_name)

# ディレクトリが存在しない場合には、作成する
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# ファイルを開いて内容を書き込む
with open(file_path, "w") as f:
    f.write(yaml_content)

print(f"ファイルを保存しました: {file_path}")


In [ ]:
#@title カメラ/ポーズを詰め合わせたyamlファイルの用意
import yaml

# 背景用yamlファイルの内容
yaml_content = '''
カメラ:
  見下ろす: from above
  見上げる: from below
  斜めから: dutch angle
  側面から: from side
  キャラの横顔: profile
  後ろから: from behind
  カメラから: viewfinder
  部分的: wide shot
  ごく部分的: very wide shot

目線や視線:
  振り返る: looking back
  こちらを見る: looking at viewer
  見上げる: looking up
  下を見る: looking down
  誰かを見ている: looking at another
  視線が何かに: looking to the side
  視線をそらす: looking away
  遠くを見る: looking afar
  真っ直ぐ見る: looking ahead

通常:
  立っている: standing
  前かがみ: leaning forward
  背筋を伸ばす: arched back
  格闘の構え: fighting stance
  もたれかかる: leaning back

運動:
  歩き: walking
  走り: running
  ジャンプ: jumping
  バランス: balancing
  スポーツ的: flyings weatdrops
  逆立ち: handstand

座る:
  座っている: sitting
  膝まづく: kneeling
  しゃがむ: squatting
  片膝立ち: one knee
  体育すわり: hugging own legs
  あぐら: indian style
  正座: seiza
  ぺたんこ座り: wariza
  横座り: yokozuwari
  四つん這い: all fours
  椅子にもたれかかって: reclining
  
寝る:
  横たわっている: lying
  寝ている: sleeping
  ベッドの上: on bed
  うつ伏せ: on stomach
'''

# ファイルパス
dir_path = "/content/stable-diffusion-webui/extensions/sdweb-easy-prompt-selector/tags"
file_name = "カメラとポーズ.yml"
file_path = os.path.join(dir_path, file_name)

# ディレクトリが存在しない場合には、作成する
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# ファイルを開いて内容を書き込む
with open(file_path, "w") as f:
    f.write(yaml_content)

print(f"ファイルを保存しました: {file_path}")


In [ ]:
#@title 個人用プロンプトを詰め合わせたyamlファイルの用意
import yaml

# 背景用yamlファイルの内容
yaml_content = '''
服装:
  メイド:(maid:1.2), gothic lolita, crossdressing, detailed dress, detailed maid apron, detailed maid skirt, detailed frills, (rem (re:zero):0.8), hair ornament, gloves
  軍人風:(gothic lolita:1.1), (white shirt:1.3), military blue jacket, military black hat, crossdressing
  サイバーパンク制服: (half-human, half-robot heroine:1.2), (school uniform:1.2), wearing jacket, crossdressing, mechanical headgear
  魔女:witch, large witch hat, black clothes, meidum breasts, fantasy
  研究者の白衣風:(scientist girl wear white labcoat:1.2), white coat, (white gloves:1.1)
'''

# ファイルパス
dir_path = "/content/stable-diffusion-webui/extensions/sdweb-easy-prompt-selector/tags"
file_name = "個人用プロンプト.yml"
file_path = os.path.join(dir_path, file_name)

# ディレクトリが存在しない場合には、作成する
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# ファイルを開いて内容を書き込む
with open(file_path, "w") as f:
    f.write(yaml_content)

print(f"ファイルを保存しました: {file_path}")


In [ ]:
#@title  modelとVAEのダウンロード
# Stable Diffusion
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.ckpt
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/raw/main/v2-inference-v.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.yaml
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt -d /content/stable-diffusion-webui/embeddings -o dblx.pt
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Conflictx/AnimeScreencap/resolve/main/AnimeScreenCap.pt -d /content/stable-diffusion-webui/embeddings -o AnimeScreenCap.pt

# AbyssOrangeMix2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AbyssOrangeMix2_hard.safetensors
# AbyssOrangeMix3
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A1_Flat,animelike.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A1B_illustration_between_A1andA3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A2_Artistic,Oil_paintings_like.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A3_Midpoint_of_Artistic_and_Kawaii.safetensors
# AbyssOrangeMix用VAE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/VAE -o OrangeMix.vae.pt

# Counterfeit-V2.5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Counterfeit-V2.5.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -d /content/stable-diffusion-webui/models/VAE -o Counterfeit-V2.5.vae.pt

# Pastel-Mix
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt -d /content/stable-diffusion-webui/models/VAE -o pastelmix-fp16.vae.pt

# SukiyakiMix
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Vsukiyaki/SukiyakiMix-v1.0/resolve/main/SukiyakiMix-v1.0-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o SukiyakiMix-fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/VAE -o SukiyakiMix-fp16.vae.pt

# Defmix-v2.0
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Defpoint/Defmix-v2.0/resolve/main/Defmix-v2.0.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Defmix-v2.0.safetensors

# blue pencil
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/bluepen5805/blue_pencil/resolve/main/blue_pencil-v5.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o blue_pencil-v5.safetensors

# Anything v5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/30163 -d /content/stable-diffusion-webui/models/Stable-diffusion -o AnythingV5.safetensors


# Waifu Diffusion 
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/waifu-diffusion/wd-1-5-beta2/resolve/main/checkpoints/wd-1-5-beta2-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd15-beta1-fp16.safetensors
# !wget https://huggingface.co/waifu-diffusion/wd-1-5-beta2/resolve/main/checkpoints/wd-1-5-beta2-fp16.yaml -O /content/stable-diffusion-webui/models/Stable-diffusionwd15-beta1-fp16.yaml.mp3
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o kl-f8-anime2.vae.pt

In [ ]:
#@title LoRAやControlNetもダウンロード

# Honkai 3rd Bronya Herrscher of Truth/崩壞3rd 布洛妮婭 真理之律者
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/27192 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o honkaiImpact3Herrscher.safetensors
# LightningVFX - Create more consistent Lightning
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12144 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o LightningVFX.safetensors
# Bondage Suspension
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/10347 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Bondage_Suspension.safetensors
# Milking machine
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/10069 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Milking_machine.safetensors
# Manhattan Cafe (umamusume)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/8935 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Manhattan_Cafe.safetensors
# Vibrator
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12843 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Vibrator.safetensors
# Esthetic Futanari + Trap
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/17951 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Futanari_Trap.safetensors
# Silence Suzuka (umamusume)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/13513 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Silence_Suzuka.safetensors

# ControlNetが使えるように、ファイルをダウンロード
"""
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth
"""

In [ ]:
#@title  起動！</br>(辛抱強く5分位待つと、最後から1つ前の行にPublic WebUI Colab remote.moe URLとしてリンクが表示されるので、それをクリックして下さい。)</br>(閉じるときは、ランタイム→ランタイムを接続解除して削除。)
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt

# 設定を予め変更する
import json
config = {
    "CLIP_stop_at_last_layers": 2,
    "quicksettings": "sd_model_checkpoint,sd_vae,CLIP_stop_at_last_layers"
}
with open("config.json", "w") as f:
    json.dump(config, f)

# 起動！！！ --no-half-vae
!python launch.py --listen --xformers --enable-insecure-extension-access --gradio-queue --multiple

# **日本語化について**
webuiは頻繁に機能が追加・更新されるので、それに追いつけてない不完全な日本語化パッチを当てるより英語のままのほうがググりやすくて便利な気がします。</br>
よって日本語化をしていません。ご容赦ください。</br>
「webui向けの拡張機能・追加ファイルの導入」に、日本語化パッチのインストール準備自体はしてあるので、コメントアウトのための#を消してインストールすれば、簡単に日本語化出来ます。

---

# **VAEとは**</br>
主に塗りに影響を与える、調整ファイル。</br>
これが不適切だと変に淡い塗りの画像が出力される。</br>
一応、AbyssOrangeMix以外は、設定をしなくても上手く動くようにしてあるはず。

---

# **導入モデルの簡易紹介**</br>
モデルの配布リンク先を見れば、プロンプトや使用Samplerが載ってる作例が見れるので、見てみてね。</br></br>
**Stable Diffusion**</br>
最も有名な画像生成AIの1つにして、様々な画像生成AIの大本になっている、Stable Diffusionの最新版モデル。</br>
リアル画像に強く、アニメ調イラストは弱い。</br>
プロンプトの書き方が、以下に記されているアニメ調イラストの生成に特化したモデルたちと違うので注意が必要。</br>
解説サイト↓</br>
https://webbigdata.jp/post-14827/</br>
</br>
**AbyssOrangeMix**：流行ってるモデル。AOM3系が最新。</br>
ややAI臭いが美麗。</br>
R-18も可能。</br>
(このモデルは、VAEを自分で選択してもらう必要があるから、注意!)</br>
推奨Negative：(worst quality, low quality:1.4), (realistic, lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0),, (abs, muscular, rib:1.0), (depth of field, bokeh, blurry:1.4), (greyscale, monochrome:1.0), text, title, logo, signature</br>
推奨Sampler：DPM++ SDE Karras</br>
https://huggingface.co/WarriorMama777/OrangeMixs</br>
</br>
**Counterfeit**-V2.5：崩れづらい分使い手の個性を出しづらい。</br>
でも、キャラクターの顔が非常に崩れにくく、衣装や小物から背景まで、全てを上手に出力してくれる。</br>
個人的には、イラスト風味の背景を描くのはこのモデルが一番上手だと思う。</br>
R-18は一応は可能。</br>
推奨Negative：EasyNegative, (worst quality, low quality:1.4)</br>
推奨Sampler: DPM++ 2M Karras</br>
推奨CFG Scale：10 ←デフォの7より高めるとパキっと濃くなる</br>
https://huggingface.co/gsdf/Counterfeit-V2.5</br>
</br>
**Pastel-Mix**：パステルカラーの独特なイラストを生成できる。AI臭さが非常に低い、個性派モデル。</br>
R-18は厳しいけど、このモデルにそんなことを期待するのもどうかと思う。</br>
推奨Sampler：DPM++ 2M Karras</br>
https://civitai.com/models/5414/pastel-mix-stylized-anime-model</br>
</br>
**SukiyakiMix**：Pstel-MixとAbyssOrangeMixを6:4でマージしただけのモデル。単純だけど、それだけに使い勝手が良い。</br>
推奨Negative：Negative:
(low quality, worst quality)1.4, (bad anatomy)+, (inaccurate limb)1.3,bad composition, inaccurate eyes, extra digit,fewer digits,(extra arms)1.2,logo,text</br>
https://huggingface.co/Vsukiyaki/SukiyakiMix-v1.0</br>
</br>
**Waifu Diffusion**</br>
法的にクリーンなモデル。現在最新は1.5beta。</br>
正直なところ、他のモデルよりも良い絵を出すのが難しいが、不正流出モデルの派生なんぞが覇権を取り続ける状況は好ましくないから、応援したい。</br>


---

# **プロンプト参考リンク**
**・使いたいモデルの配布サイト**</br>
作者が直々に看板として作った上手な作例が載ってるので、大いに参考にすべし。</br>
</br>
**・【NovelAI】 エロもいける！最強プロンプト（呪文）辞典**</br>
**・【ローカル版】エロ特化！最強プロンプト（呪文）新辞典★無料公開版★**</br>
この名前だけど、エロ目的じゃなくても非常に参考になる実践プロンプト集になっている。</br>
https://iosworld.blog.2nt.com/blog-entry-528.html</br>
https://iosworld.blog.2nt.com/blog-entry-558.html</br>
</br>
**・ちちぷい**</br>
AIイラスト版のPixiv。投稿作の多くにモデルやプロンプト、サンプラーが載っているので、参考になる。</br>
https://www.chichi-pui.com</br>
</br>
**・MajinAI**</br>
AIイラスト投稿サイト2。こっちはイラスト生成者向けのプロンプト紹介サイト。</br>
https://majinai.art/ja/index.php</br>
</br>
**・【随時更新】NovelAI/Waifu Diffusionを使う際に有用なサイト・リンク集(備忘録)**</br>
更新頻度・速度ともに高いサイト。</br>
https://economylife.net/links-for-novelai-waifu-diffusion/#gsc.tab=0


---



# **プロンプトを組む上でのプチアドバイス**
**・トークンの区切り**</br>
Stable Diffusionは、一度に75トークンまでしか扱えないので、長いプロンプトを指示すると分割されて実行されます。</br>
その際、プロンプトが断裂して（例えばsilver hairが、silverとhairに分けられて銀髪にならなくなって）しまったり、次の区切りの76トークン目が必要以上に強く影響したり、といった現象が発生します。</br>
これを防ぐために「BREAK」というワードを使って強制的にプロンプトを区切ることができます。</br>
　　例：1girl, silver hair, **BREAK**, hair ornament, ……</br>
BREAK直後のタグは強めに影響することを意識すると、より意図した絵を出しやすくなります。</br>
</br>
**・惜しい絵が出たとき、その細部を変える**</br>
Seedの隣りにあるExtraをチェックすると出てくる、VariationSeed・variation strengthの値をいじって再生成することで、その惜しい絵を微妙にズラすことが出来ます。
「variation seed」に惜しかった絵のSeed値を入れ、どれくらいそこからズラすかをvariation strengthで決めます。
0.1で様子見をして、強さを上下させて調整してみましょう。

**・惜しい絵が出たとき、その一部を変える**
出たイラストをSend to inpaint に送って、変更したい部分を黒塗りにして生成しましょう。</br>
もしくは、画像をダウンロードして変更したい部分を雑に色塗りして直して、それをinpaintにアップロードして変更箇所を黒塗りにして、生成しましょう。

---



# **Textual Inversion</br>**
わざと駄目なイラストを学習させたモデルをネガティブプロンプトから呼ぶことで、品質を向上させる技術があります。</br>
このコードではEasyNegativeというものを導入済みなので、ネガティブプロンプトの欄に**EasyNegative**と打つだけで、**強力な品質向上**が狙えます。</br>
Counterfeitというモデルの作者が作ったものなので、それと相性が良いです。デメリットとしては、ちょっと**リアル寄りの画風に引っ張られる節**もあります。</br>
bad_prompt_version2というものも入れてあるので、これも使えますが、EasyNegativeよりも絵柄への影響が強いです。

---



# **ControlNet**
話題のControlNetは導入済みなのですぐ使えます。</br>

---



# **LoRA**</br>
既存モデルに、なるべく低負荷で、画風やポーズ、特定キャラクターなどの追加要素を学ばせる仕組みとしてLoRAというものがあります。</br>
試験的に、</br>
・[崩壊3rdの真理の律者のデザインを追加するLoRAモデル](https://civitai.com/models/9101/honkai-impact-3-herrscher-of-truth)</br>
・[電撃を放てるようになるLightningVFX](https://civitai.com/models/10222/lightningvfx-create-more-consistent-lightning)</br>
などを組み込んでおきました。</br>
additional networkをEnableにして、モデルを選んで、weightを決めて、トリガーワードをプロンプトに組み込めば、使えます。</br>
コードをいじれば、他のLoRAモデルも導入できると思うので、興味があったら私に連絡するか、自分で頑張ってみてください。</br>
weightをどう設定するか迷ったら、とりあえず0.6(～0.8)くらいにしておくと良いよ。</br>
</br>
※LoRAが配布されているサイト</br>
https://civitai.com</br>
絞り込めばLoRAだけを見れます。</br>

| Loraモデル | トリガーになるプロンプト | 推奨weight |
| - | - | - |
| [真理の律者](https://civitai.com/models/22769/honkai-3rd-bronya-herrscher-of-truth3rd) | 1girl,herrscher of truth,halo, grey hair,grey eyes,ponytail,hair ornament,small breasts,dress,gloves,thigh boots or white thighhighs |  |
| [LightningVFX](https://civitai.com/models/10222/lightningvfx-create-more-consistent-lightning) | Shooting lightning from his hands, Holding a lightning ball in his hand, Surrounded by lighting, Lightning glowing, Tiny spark lightning, </br>Particle glow , Intricate line of purple lightning , Rays straight line , Bloom , Particle lightning , Lightning bolts , Lightning bolt sword in hand </br>Purple/network/blue/yellow lightning | 0.2 |
| [Manhattan Cafe (umamusume)](https://civitai.com/models/7607/manhattan-cafe-umamusume) | manhattan cafe \(umamusume\), white ahoge, black long hair</br>Racing competition clothes :black coat, black shirt, black skirt, yellow necktie | 0.5~0.65 |
| [Silence Suzuka (umamusume)](https://civitai.com/models/11415/silence-suzuka-umamusume)</br>NOT COMPATIBLE WITH AOM3A2, AOM3A3 model. | silence suzuka \(umamusume\)</br>black gloves, white skirt, pleated skirt, black pantyhose, layered sleeves, puffy short sleeves, short over long sleeves, black bow, mismatched footwear, asymmetrical footwear</br>(optional) smile. she is serious when running</br> | 0.7~1.0 |

<details>
  <summary>R-18のLoRA</summary>

  | Loraモデル | トリガーになるプロンプト | 推奨weight |
  | - | - | - |
  | [Bondage Suspension](https://civitai.com/models/8762/bondage-suspension) | suspension, ball gag, arms behind back |  |
  | [Milking machine](https://civitai.com/models/8500/milking-machine) | milking machine, lactation, nipples, leaning forward, |  |
  | [Vibrator](https://civitai.com/models/10820/vibrator) | vibrator on nipple, vibrator under clothes, vibrator in thighhighs, vibrator cord, egg vibrator, tape |  |
  | [Esthetic Futanari + Trap](https://civitai.com/models/7327/esthetic-futanari-trap-v37) | penis, futanari, otoko no ko, female focus, small penis, large penis (carefully, but work), medium penis| 0.3-.0.6, 0.7 |
  
</details>

# **便利な設定**</br>
自動で設定されるようにプログラムしておきました。</br>
</br>
~~**settings →Userinterface →Quicksetting list**（下の方にある） に</br>
sd_model_checkpoint,sd_vae,CLIP_stop_at_last_layers</br>
と書いて、</br>
**Apply settings →Reload UI**</br>
と押すと、UIの一番からVAEが変えられるようになります。</br>
エラーが起きますが、最後の起動コードを停止■させ、再度実行▶すれば大丈夫です。待ち時間も2回目は短いです。</br>
Clip Skipはまあ2にしておいて。俺もよくわからない。~~</br>

---